In [1]:
from itertools import chain

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm


import torch
import spacy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

/Users/elnardu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# initialize hyper hyper parameters
torch.manual_seed(420)
np.random.seed(420)

In [3]:
import gensim
import pickle

In [ ]:
# NOTE -- You just need to provide your own path to a w2v binary here if you don't have it
# we used the one from our NLP Course's 2nd HW Assignment
wv_from_bin = gensim.models.KeyedVectors.load_word2vec_format("../data/w2v.bin", binary=True)

# Load data

In [4]:
fallacies_df = pd.read_csv('../data/fallacies.csv', index_col=0)
approved_df = pd.read_csv('../data/approved.csv', index_col=0)

In [5]:
df = pd.concat([
    fallacies_df,
    approved_df[approved_df.n_supporters >= 5].drop("n_supporters", axis=1)
])
df.fallacy_reason = df.fallacy_reason.fillna('')
df = df[~df.premise_content.isna()]

In [6]:
vc = df.fallacy_type.value_counts()

In [7]:
df = df[df.fallacy_type.isin(vc.head(10).index)]

# Text preprocessing

In [8]:
nlp = spacy.load('en_core_web_sm')

I0509 21:43:38.169584 4684856768 file_utils.py:39] PyTorch version 1.0.1.post2 available.
I0509 21:43:38.207648 4684856768 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [9]:
def preprocess_sentence(sent):
    sent = sent.lower()
    sent = nlp(sent)
    words = map(lambda x: x.text, sent)
    return list(words)

In [10]:
df['premise_content_preprocessed'] = df.premise_content.apply(preprocess_sentence)

In [11]:
df = df.sample(frac=1, random_state=420).reset_index(drop=True) # shuffle the data

In [12]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=420)

In [13]:
word_vocab = set(chain.from_iterable(map(lambda x: x[1]["premise_content_preprocessed"], train_df.iterrows())))
sorted_words = sorted(list(word_vocab))

Skip this whole part below if you already have calculated the needed vectors and word map ( we will save and reload them in this notebook, but this could be useful if you were trying to replicate our work and just needed the vectors we have in our ../data directory!).

In [ ]:
word_to_ix = {}
# we need to create a mapping with all the vectors we might need!
needed_vectors = []
for i in range(len(sorted_words)):
    try:
        # Order is important, if we found the word, the we'll append it
        # if we haven't we'll consider this an oov word!
        needed_vectors.append(wv_from_bin[sorted_words[i]])
        word_to_ix[sorted_words[i]] = len(needed_vectors)
    except KeyError:
        # this will be our OOV term
        word_to_ix[sorted_words[i]] = 0


#OOV and PAD vectors are both random
needed_vectors.insert(0, np.random.randn(*needed_vectors[0].shape))
needed_vectors.append(np.random.randn(*needed_vectors[0].shape))
word_to_ix['<oov>'] = 0
word_to_ix['<pad>'] = len(needed_vectors) - 1

needed_vectors = torch.FloatTensor(needed_vectors)
torch.save(needed_vectors,'../data/w2v_needed_vectors.pt')
# Also make sure to save the word_map
print(needed_vectors.shape)
f = open("../data/w2v_word_to_ix.pkl", "wb")
pickle.dump(word_to_ix, f)
f.close()

In [20]:
needed_vectors = torch.load('../data/w2v_needed_vectors.pt')
with open("../data/w2v_word_to_ix.pkl", 'rb') as fr:
    word_to_ix = pickle.load(fr)

In [21]:
fallacy_vocab = sorted(list(set(df.fallacy_type.unique())))
fallacy_to_ix = {word: i for i, word in enumerate(fallacy_vocab)}

# Neural Net

In [22]:
class Net(nn.Module):
    def __init__(self, word_vocab_size, word_embedding_dim, fallacy_vocab_size, max_sent_size, word2vec_tensor):
        super(Net, self).__init__()
        
        self.word_embeddings = nn.Embedding.from_pretrained(word2vec_tensor)

        hidden = 100
        self.fc1 = nn.Linear(word_embedding_dim, hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden * max_sent_size, fallacy_vocab_size)
        
    def forward(self, word_inputs):
        word_embeds = self.word_embeddings(word_inputs)
        h1 = self.fc1(word_embeds)
        a1 = self.relu(h1)
        h2 = self.fc2(a1.view(a1.shape[0], -1))
        return h2

In [23]:
MAX_SENT_SIZE = 100

In [24]:
def pad_and_convert_to_ints(data):
    X = np.full((len(data), MAX_SENT_SIZE), word_to_ix["<pad>"])

    for i, (_, x) in enumerate(data.iterrows()):
        X[i, :len(x["premise_content_preprocessed"])] = [
            (word_to_ix[word] if word in word_to_ix else word_to_ix['<oov>'])
            for word in x["premise_content_preprocessed"]
        ]

    return X

In [25]:
trainX = pad_and_convert_to_ints(train_df)
testX = pad_and_convert_to_ints(test_df)

In [26]:
trainY = train_df.fallacy_type.apply(lambda x: fallacy_to_ix[x]).values
testY = test_df.fallacy_type.apply(lambda x: fallacy_to_ix[x]).values

In [27]:
net = Net(
    len(word_vocab),
    300,
    len(fallacy_vocab),
    MAX_SENT_SIZE,
    needed_vectors
)
opt = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss()

In [28]:
def train_epoch(X, Y, model, opt, criterion, batch_size=50):
    model.train()
    losses = []
    for beg_i in range(0, X.shape[0], batch_size):
        x_batch = X[beg_i : beg_i + batch_size, :]
        y_batch = Y[beg_i : beg_i + batch_size]
        x_batch = torch.tensor(x_batch)
        y_batch = torch.tensor(y_batch)

        opt.zero_grad()

        y_pred = model(x_batch)

        loss = criterion(y_pred, y_batch)

        loss.backward()
        
        opt.step()

        losses.append(loss.data.numpy())

    return [sum(losses) / float(len(losses))]

In [29]:
e_losses = []
num_epochs = 50
for e in tqdm(range(num_epochs)):
    e_losses += train_epoch(trainX, trainY, net, opt, criterion, batch_size=100)

100%|██████████| 50/50 [00:14<00:00,  3.42it/s]


In [30]:
e_losses

[1.5718819043215584,
 1.3443750143051147,
 1.2172827089534086,
 1.1182001829147339,
 1.0266545081839842,
 0.9365077194045571,
 0.8579689334420597,
 0.7879927877117606,
 0.7204733017612907,
 0.6607672957813039,
 0.6102612526977763,
 0.5645940680714214,
 0.5198978097999797,
 0.47869546141694574,
 0.4450809333254309,
 0.40894194560892444,
 0.38236001440707373,
 0.35993782650021944,
 0.3378117001231979,
 0.317760699592969,
 0.3012916390948436,
 0.28519582638845725,
 0.26968511829481406,
 0.256101849324563,
 0.24312194787404118,
 0.23442038214382002,
 0.22222944480531356,
 0.2150202073595103,
 0.2054996578132405,
 0.1983595446628683,
 0.19052798235241106,
 0.18322673396152608,
 0.17939207834355972,
 0.17480437720523162,
 0.16789642173577757,
 0.1650044146706076,
 0.1601428900133161,
 0.15652185678482056,
 0.15114454793579438,
 0.15185996605192914,
 0.14738256834885655,
 0.14671821979915395,
 0.14296554752132473,
 0.14055847639546676,
 0.14026852740960963,
 0.13660823268925443,
 0.1352916254

# Metrics (Macro F1 Score)

In [31]:
with torch.no_grad():
    net.eval()
    x = torch.tensor(testX)
    y_pred = net(x)

In [32]:
score = f1_score(testY, y_pred.numpy().argmax(axis=1), average='macro')

/Users/elnardu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
score

0.1990570051360325

In [34]:
for i in range(10):
    print("Guessing " + fallacy_vocab[i] + "\t for all:", end='\t')
    print(f1_score(testY, np.array( [i]*testY.shape[0]), average='macro'))

Guessing Appeal To Authority	 for all:	0.004347826086956523
Guessing Appeal To Belief	 for all:	0.0064516129032258064
Guessing Begging The Question	 for all:	0.005405405405405406
Guessing Fallacy Of False Cause	 for all:	0.00851063829787234
Guessing Fallacy Of Red Herring	 for all:	0.005405405405405406
Guessing Irrelevant Conclusion	 for all:	0.02178217821782178
Guessing None	 for all:	0.07959866220735787
Guessing Poisoning The Well	 for all:	0.0011049723756906078
Guessing Prejudicial Language	 for all:	0.003278688524590164
Guessing Wrong Direction	 for all:	0.0074866310160427805


/Users/elnardu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/elnardu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/elnardu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/elnardu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/elnardu/anaco

In [35]:
print("Random Guessing:", end='\t')
print(f1_score(testY, np.random.randint(0,10,testY.shape[0]), average='macro'))

Random Guessing:	0.02837729816147082
